In [1]:
from numpy import loadtxt
import math
import copy
import csv

In [2]:
def parent(i):
    '''
    This function finds the index of parrent of given index i
    
    Inputs:
            i: given index i
    Output:
            parent_index: index of parent of i
    '''
    if i == 0: # for the root
        parent_index = 0
    elif i%2 == 0:
        parent_index = int(i/2-1)
    else:
        parent_index = int(math.floor(i/2)) # for python
    
    return parent_index


def children(i):
    '''
    This function finds children of given index i
    '''
    return (2*i+1,2*i+2)


def insert_to_heap(H,x,i):
    '''
    This function inserts a new key into a given heap
    
    Inputs: 
            H: current given heap
            x: given new number to be inserted into H (the key in the heap)
            i: a value associated with key x
    Outputs:
            H: updated H adfter insertation
    '''
    n = len(H)
    H.append([x,i]) 
    index_x = n # initial index of X is n (last element)
    check = True
    while check == True:
        prent_index = parent(index_x)
        if H[index_x][0] < H[prent_index][0]:
            # swap H[index_x] and H[prent_index]
            c = H[index_x]
            H[index_x] = H[prent_index]
            H[prent_index] = c
            index_x = prent_index #update x index
        else:
            check = False
    return H

def extract_min_heap(H):
    '''
    This function extract minimum of given heap
    
    Inputs: 
            H: current heap
    Outputs:
            min_value: the minimum value of heap
            H: updated heap after extraction of minimum value
    '''    
    n = len(H)
    min_value = H[0]
    last_leaf = H.pop(n-1) 
    if len(H)>0: # check if after removal of min, the heap is not an empty list
        H[0] = last_leaf
    else:
        check = False
    
    check = True
    index_x = 0              
    while check == True:
        try: # applies when we don't reach to a leaf
            try: # aplies when the node has two children
                (c1_index,c2_index) = children(index_x) 
                # find the index of smaller child
                if H[c1_index][0] < H[c2_index][0]: 
                    child_index = c1_index
                else:
                    child_index = c2_index 
                if H[index_x][0] > H[child_index][0]:
                        # swap H[index_x] and H[child_index]
                        c = H[index_x]
                        H[index_x] = H[child_index]
                        H[child_index] = c
                        index_x = child_index #update x index
                else:
                    check = False
            except: # applies when the node has one child
                (c1_index,c2_index) = children(index_x) 
                child_index = c1_index 
                if H[index_x][0] > H[child_index][0]:
                        # swap H[index_x] and H[child_index]
                        c = H[index_x]
                        H[index_x] = H[child_index]
                        H[child_index] = c
                        index_x = child_index #update x index
                else:
                    check = False

        
        except: # applies when we reach to a leaf (the node has no child)
            check = False   
    
    return H, min_value

def delet_from_heap(H,x):
    '''
    This function delets given vortex, x, from a heap. Given vortex of x is the first element in the elements
    of the heap, e.g. x = 45 deletes, [45, 23, -5423] where -5423 is the key in the heap.
    
    Inputs: 
            H: current heap
    Outputs:
            H: updated heap after deletion of x
    '''    
    n = len(H)
    
    if H[-1][0] == x: # if x is the last element of the heap.
        H.pop(n-1) 
    else:
        last_leaf = H.pop(n-1) 
        check1 = True
        check2 = 1

        # finding index of deleted item in the heap 
        for i in range(0,len(H)):
            if H[i][0] == x:
                index_x = i
                break

        H[index_x] = last_leaf

        # check with parent 
        while check1 == True:
            prent_index = parent(index_x) 
            if H[index_x][2] < H[prent_index][2]:
                # swap H[index_x] and H[child_index]
                c = H[index_x]
                H[index_x] = H[prent_index]
                H[prent_index] = c
                index_x = prent_index #update x index
                check2 += 1
            else:
                check1 = False


        while check2 == 1:
            try: # applies when we don't reach to a leaf


                try: # aplies when the node has two children
                    (c1_index,c2_index) = children(index_x) 
                    # find the index of smaller child
                    if H[c1_index][2] < H[c2_index][2]: 
                        child_index = c1_index   
                    else:
                        child_index = c2_index 
                except: # applies when the node has one child
                    (c1_index,c2_index) = children(index_x) 
                    child_index = c1_index
                    
                if H[index_x][2] > H[child_index][2]:
                    # swap H[index_x] and H[child_index]
                    c = H[index_x]
                    H[index_x] = H[child_index]
                    H[child_index] = c
                    index_x = child_index #update x index
                else:
                    check2 = 2

            except: # applies when we reach to a leaf (the node has no child)
                check2 = 2   
    
    return H

In [3]:

def huffman(H):
    '''
    Given a heap based on probability of the items, creats tree T based on Huffman algorithm
    
    Inputs:
            H: given heap based on probability of items (root lowest probability of items)
            
    Outputs:
            T: a dictionary where items are items and values are the codework of the items
    '''
    if len(H) == 2:
        H, a = extract_min_heap(H)
        H, b = extract_min_heap(H)
        T = {a[1]:'0', b[1]:'1'}
        
    else:
        H, a = extract_min_heap(H)
        H, b = extract_min_heap(H)
        p_ab = a[0] + b[0]
        insert_to_heap(H,p_ab,b[1]+'_'+a[1]) # insert ab into heap
        T = huffman(H)
        T = extension(T,b[1],a[1])
    return T

def extension(T,b,a):
    '''
    This function update tree T by extending the node "ba" in to two new nodes b and a
    
    Inpust:
            T: is given tree T that has a node "ba". T is dictionary where the items (numbers/alphabets)
            are keys and code work of the nodes are the values.
            b and a: are strings (e.g. b = 4_5_2 and a = 8_6)
    
    Outputs:
            T: updated tree where a and b are two seperate nodes after deviding node "ba"
    '''
    # T = {'x':xx, 'a_b':yy, 'z':zz}
    #  T = {'x':xx, 'b':yy+'0', 'a':yy+'1', 'z':zz}
    
    yy = T[b+'_'+a] # codework of ba
    T.pop(b+'_'+a, None) # deleting ba from tree
    before_ = b # b node
    after_ = a # a node
    T[before_] = yy + '0' # add 0 to the left of code work ba for b
    T[after_] = yy + '1' # add 1 to the left of code work ba for a
    
    return T

In [12]:
f=open('huffman_Test_01_min_2_max_5.txt',"r")
lines=f.readlines()
numbers_list=[]
for x in lines:
    numbers_list.append(int(x))
f.close()

n = numbers_list.pop(0)

# create the heap
H = []
s = 1
for weight in numbers_list:
    insert_to_heap(H,weight,str(s))
    s = s+1
    
T = huffman(H)


print('\n min length of codeswork')
min_length = min([len(v) for k,v in T.items()])
print(min_length)
print('\n max length of codeswork')
max_length = max([len(v) for k,v in T.items()])
print(max_length)

# sort T based on its key (key of T are the numbers/alphabets or the row number in the input file)
# change key of T to int
T = {int(k):v for k,v in T.items()}
T_sorted_numbers = dict(sorted(T.items())) # sort based on numbers/alphabet/rows
print('\n codework of all numbers/alphabets (row number in the input file)')
print(T_sorted_numbers)

# add the numbers/alphabets/rows weight to T
for i in range(0,len(numbers_list)):
    T_sorted_numbers[i+1] = [T_sorted_numbers[i+1], numbers_list[i]]

T_sorted_weights = dict(sorted(T.items(), key=lambda item: item[1][1])) # sort based on weights
print('\n codeworks, of all numbers/alphabets with their associated weights')
print(T_sorted_weights)


 min length of codeswork
2

 max length of codeswork
5

 codework of all numbers/alphabets (row number in the input file)
{1: '1011', 2: '001', 3: '1010', 4: '00000', 5: '0001', 6: '100', 7: '01', 8: '111', 9: '00001', 10: '110'}

 codeworks, of all numbers/alphabets with their associated weights
{6: '100', 2: '001', 3: '1010', 1: '1011', 5: '0001', 4: '00000', 9: '00001', 7: '01', 10: '110', 8: '111'}


In [10]:
f=open('huffman_Test_02_min_3_max_6.txt',"r")
lines=f.readlines()
numbers_list=[]
for x in lines:
    numbers_list.append(int(x))
f.close()

n = numbers_list.pop(0)

# create the heap
H = []
s = 1
for weight in numbers_list:
    insert_to_heap(H,weight,str(s))
    s = s+1
    
T = huffman(H)


print('\n min length of codeswork')
min_length = min([len(v) for k,v in T.items()])
print(min_length)
print('\n max length of codeswork')
max_length = max([len(v) for k,v in T.items()])
print(max_length)

# sort T based on its key (key of T are the numbers/alphabets or the row number in the input file)
# change key of T to int
T = {int(k):v for k,v in T.items()}
T_sorted_numbers = dict(sorted(T.items())) # sort based on numbers/alphabet/rows
print('\n codework of all numbers/alphabets (row number in the input file)')
print(T_sorted_numbers)

# add the numbers/alphabets/rows weight to T
for i in range(0,len(numbers_list)):
    T_sorted_numbers[i+1] = [T_sorted_numbers[i+1], numbers_list[i]]

T_sorted_weights = dict(sorted(T.items(), key=lambda item: item[1][1])) # sort based on weights
print('\n codeworks, of all numbers/alphabets with their associated weights')
print(T_sorted_weights)


 min length of codeswork
3

 max length of codeswork
6

 codework of all numbers/alphabets (row number in the input file)
{1: '000', 2: '101111', 3: '100000', 4: '110', 5: '0101', 6: '001', 7: '100001', 8: '1010', 9: '101110', 10: '011', 11: '1001', 12: '10110', 13: '0100', 14: '10001', 15: '111'}

 codeworks, of all numbers/alphabets with their associated weights
{1: '000', 6: '001', 11: '1001', 8: '1010', 14: '10001', 12: '10110', 3: '100000', 7: '100001', 9: '101110', 2: '101111', 4: '110', 15: '111', 10: '011', 13: '0100', 5: '0101'}


In [11]:
f=open('huffman.txt',"r")
lines=f.readlines()
numbers_list=[]
for x in lines:
    numbers_list.append(int(x))
f.close()

n = numbers_list.pop(0)

# create the heap
H = []
s = 1
for weight in numbers_list:
    insert_to_heap(H,weight,str(s))
    s = s+1
    
T = huffman(H)


print('\n min length of codeswork')
min_length = min([len(v) for k,v in T.items()])
print(min_length)
print('\n max length of codeswork')
max_length = max([len(v) for k,v in T.items()])
print(max_length)

# sort T based on its key (key of T are the numbers/alphabets or the row number in the input file)
# change key of T to int
T = {int(k):v for k,v in T.items()}
T_sorted_numbers = dict(sorted(T.items())) # sort based on numbers/alphabet/rows
print('\n codework of all numbers/alphabets (row number in the input file)')
print(T_sorted_numbers)

# add the numbers/alphabets/rows weight to T
for i in range(0,len(numbers_list)):
    T_sorted_numbers[i+1] = [T_sorted_numbers[i+1], numbers_list[i]]

T_sorted_weights = dict(sorted(T.items(), key=lambda item: item[1][1])) # sort based on weights
print('\n codeworks, of all numbers/alphabets with their associated weights')
print(T_sorted_weights)


 min length of codeswork
9

 max length of codeswork
19

 codework of all numbers/alphabets (row number in the input file)
{1: '011100110', 2: '1001000111', 3: '10011101101', 4: '010011010', 5: '11000001010', 6: '11000101111', 7: '1000101110', 8: '1010111101', 9: '10111100110', 10: '001000110', 11: '1000010011', 12: '111111100', 13: '111111001', 14: '0011010101', 15: '1001111000110', 16: '000110010', 17: '011101010', 18: '1011110111', 19: '01100100100', 20: '0011100100', 21: '1000010001', 22: '10001100000', 23: '1010110010', 24: '001010110', 25: '10110001110', 26: '0001111110010', 27: '1100101111', 28: '1001011010101', 29: '10111001111101', 30: '1010011010', 31: '111100111', 32: '1011100101', 33: '1000101101', 34: '110011100', 35: '001000101010', 36: '1000011100', 37: '110101000', 38: '11001001101', 39: '00010101100', 40: '010010101', 41: '001001011', 42: '111101000', 43: '000100011', 44: '001000100', 45: '011000010', 46: '1101110010', 47: '11001101001', 48: '1001011100', 49: '0111101